<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/cifar10_cnn_inception1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import tensorflow as tf
import numpy as np
from keras.datasets import cifar10
from functools import partial
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape, x_test.shape
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
y_train = y_train.astype(np.int32).reshape(-1)
y_test = y_test.astype(np.int32).reshape(-1)
from sklearn.model_selection import train_test_split
x_train, x_val = train_test_split(x_train, test_size=0.2, random_state=42)
y_train, y_val = train_test_split(y_train, test_size=0.2, random_state=42)

Using TensorFlow backend.


170500096/170498071 [==============================] - 4s 0us/step


In [0]:
import time
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

def shuffle_batch(x, y, batch_size):
  rnd_idx = np.random.permutation(len(x))
  n_batch = len(x) // batch_size
  for batch_idx in np.array_split(rnd_idx, n_batch):
    xs, ys = x[batch_idx], y[batch_idx]
    yield xs, ys

name = ['drop', 'layer', 'bn', 'act']
name_list = []
for i in range(4):
  bb = [b + str(i) for b in name]
  name_list.append(bb)
  bb = []

n_layers=4
n_epochs = 101
batch_size = 1000
x = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3], name="x")
y = tf.placeholder(dtype=tf.int32, shape=[None], name="y")
training = tf.placeholder_with_default(False, shape=[], name="training")
he_init = tf.variance_scaling_initializer()

##########convolution########
conv = partial(tf.layers.conv2d, strides=(1,1), padding='SAME', activation=tf.nn.relu, kernel_initializer=he_init)
max_pool = partial(tf.layers.max_pooling2d, pool_size=2, padding='SAME', strides=2)
drop = partial(tf.layers.dropout, rate=0.5, training=training)

conv1 = conv(x, filters=64, kernel_size=5, name="conv1")
max_pool0 = tf.layers.max_pooling2d(conv1, pool_size=3, strides=2, padding='SAME', name='max_pool0')
bn1 = tf.layers.batch_normalization(max_pool0, training=training, momentum=0.9)
conv2 = conv(bn1, filters=64, kernel_size=1, name='conv2')
conv3 = conv(conv2, filters=192, kernel_size=3, name="conv3")
bn2 = tf.layers.batch_normalization(conv3, training=training, momentum=0.9)
max_pool1 = tf.layers.max_pooling2d(bn2, pool_size=3, strides=2, padding='SAME', name="max_pool1")

####shape=[]

incept1 = conv(max_pool1, filters=64, kernel_size=1, name="incept1")
incept2_1 = conv(max_pool1, filters=96, kernel_size=1, name='incept2_1')
incept2_2 = conv(incept2_1, filters=128, kernel_size=3, name='incept2_2')
incept3_1 = conv(max_pool1, filters=16, kernel_size=1, name='incept3_1')
incept3_2 = conv(incept3_1, filters=32, kernel_size=5, name='incept3_2')
incept4_1 = tf.layers.max_pooling2d(max_pool1, pool_size=3, strides=(1,1), padding='SAME', name="incept4_1")
incept4_2 = conv(incept4_1, filters=32, kernel_size=1, name='incept4_2')
inception1 = tf.concat([incept1, incept2_2, incept3_2, incept4_2], axis=3, name="inception1")

max_pool2 = tf.layers.max_pooling2d(inception1, pool_size=3, strides=2, padding='SAME')


incept12 = conv(max_pool2, filters=128, kernel_size=1, name="incept12")
incept2_12 = conv(max_pool2, filters=128, kernel_size=1, name='incept2_12')
incept2_22 = conv(incept2_12, filters=192, kernel_size=3, name='incept2_22')
incept3_12 = conv(max_pool2, filters=32, kernel_size=1, name='incept3_12')
incept3_22 = conv(incept3_12, filters=96, kernel_size=5, name='incept3_22')
incept4_12 = tf.layers.max_pooling2d(max_pool2, pool_size=3, strides=(1,1), padding='SAME', name="incept4_12")
incept4_22 = conv(incept4_12, filters=64, kernel_size=1, name='incept4_22')
inception2 = tf.concat([incept12, incept2_22, incept3_22, incept4_22], axis=3, name="inception2")
###############

avg_pool = tf.layers.average_pooling2d(inception2, pool_size=4, strides=(1,1), padding='VALID', name='avg_pool')
reshape = tf.reshape(avg_pool, shape=[-1, 480])

####fully connected#####
drop5 = tf.layers.dropout(reshape, rate=0.5, training=training, name="drop5")
layer5 = tf.layers.dense(drop5, 10, kernel_initializer=he_init, name="layer5")
################################

logits = layer5
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name="xentropy")
loss = tf.reduce_mean(xentropy, name="loss")
optimizer = tf.train.AdamOptimizer(0.001, name="optimizer")
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
init = tf.global_variables_initializer()
saver = tf.train.Saver()
cost_sum = []
file_writer = tf.summary.FileWriter("./", tf.get_default_graph())
with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for xs, ys in shuffle_batch(x_train, y_train, batch_size):
      _, cost, __ = sess.run([training_op, loss, extra_update_ops], feed_dict={x: xs, y: ys, training: True})
      cost_sum.append(cost)
    cost_avg = sum(cost_sum) / len(cost_sum)
    if epoch % 10 == 0:
      acc_val = sess.run(accuracy, feed_dict={x: x_val, y: y_val})
      print("{}번째\t손실: {}\t테스트정확도: {}".format(epoch, cost_avg, acc_val))
    cost_avg = 0.0
  save_path = saver.save(sess, "./model")
  print("걸린시간: ", time.time()-start)
# with tf.Session() as sess:
#   init.run()
#   a = conv3.eval(feed_dict={x: x_test, y: y_test})
#   print(a.shape)


Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
0번째	손실: 1.8816168278455734	테스트정확도: 0.44429999589920044
10번째	손실: 0.9428329532796687	테스트정확도: 0.7153000235557556
20번째	손실: 0.6251021330821372	테스트정확도: 0.7483999729156494
30번째	손실: 0.44971522957447074	테스트정확도: 0.7440000176429749
40번째	손실: 0.3505026534716485	테스트정확도: 0.7448999881744385
50번째	손실: 0.28782023839725585	테스트정확도: 0.73089998960495
60번째	손실: 0.24525052799182928	테스트정확도: 0.7530999779701233
70번째	손실: 0.21611062336293324	테스트정확도: 0.7455000281333923
80번째	손실: 0.19209079959905523	테스트정확도: 0.7480999827384949
90번째	손실: 0.17337904146489955	테스트정확도: 0.7373999953269958
100번째	손실: 0.1587336797528845	테스트정확도: 0.7505000233650208
걸린시간:  818.5068657398224


In [0]:
with tf.Session() as sess:
  saver.restore(sess, "./model")
  acc_test = accuracy.eval(feed_dict={x: x_test, y: y_test})
  print(acc_test)

INFO:tensorflow:Restoring parameters from ./model
0.7497


In [0]:
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)
x = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3], name="x")
init = tf.global_variables_initializer()
conv1 = tf.layers.conv2d(x, 10, kernel_size=3, strides=(1,1), padding='SAME')
# conv2 = tf.layers.conv2d(conv1, 20, kernel_size=3, strides=(1,1), padding='SAME')
# conv3 = tf.layers.conv2d(conv2, 30, kernel_size=3, strides=(1,1), padding='SAME')
with tf.Session() as sess:
  init.run()
  a = conv1.eval(feed_dict={x: x_test})
  print(a.shape)

FailedPreconditionError: ignored